## Warning: TF 2.0 Dependency

This code relies on TF 2.0, which AI Platform online prediction doesn't support yet. The code works locally, and the model deploys, but upon prediction it fails, presumably because the AI Platform nodes are running TF 1.13.

We should re-test this notebook once TF 2.0 is support for online prediction. With any luck it will work without any further changes.

In [2]:
PROJECT_ID = 'vijays-sandbox'
BUCKET = 'vijays-sandbox-ml'
MODEL_PATH = 'translate_models/baseline'
MODEL_NAME = 'translate'
VERSION_NAME = 'v1'

## 1. Upload Model Artifacts to GCS

In [18]:
!gsutil cp $MODEL_PATH/encoder_model.h5 $MODEL_PATH/decoder_model.h5 gs://$BUCKET/translate/model/
!gsutil cp $MODEL_PATH/encoder_tokenizer.pkl $MODEL_PATH/decoder_tokenizer.pkl gs://$BUCKET/translate/model/

Copying file://translate_models/baseline/encoder_model.h5 [Content-Type=application/octet-stream]...
Copying file://translate_models/baseline/decoder_model.h5 [Content-Type=application/octet-stream]...
\
Operation completed over 2 objects/63.4 MiB.                                     
Copying file://translate_models/baseline/encoder_tokenizer.pkl [Content-Type=application/octet-stream]...
Copying file://translate_models/baseline/decoder_tokenizer.pkl [Content-Type=application/octet-stream]...
-
Operation completed over 2 objects/652.3 KiB.                                    


## 2. Implement Predictor Interface

In [7]:
%%writefile predictor.py
import os
import pickle
import unicodedata
import re

import numpy as np
import tensorflow as tf

MAX_TRANSLATE_LENGTH = 11 

class TranslatePredictor(object):
    def __init__(self, encoder_model, encoder_tokenizer, 
                 decoder_model, decoder_tokenizer):
      self.encoder_model = encoder_model
      self.encoder_tokenizer = encoder_tokenizer
      self.decoder_model = decoder_model
      self.decoder_tokenizer = decoder_tokenizer

    def _unicode_to_ascii(self, s):
        return ''.join(c for c in unicodedata.normalize('NFD', s)
            if unicodedata.category(c) != 'Mn')

    def _preprocess_sentence(self, w):
        w = self._unicode_to_ascii(w.lower().strip())

        # creating a space between a word and the punctuation following it
        # eg: "he is a boy." => "he is a boy ."
        # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
        w = re.sub(r"([?.!,¿])", r" \1 ", w)
        w = re.sub(r'[" "]+', " ", w)

        # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
        w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

        w = w.rstrip().strip()

        # adding a start and an end token to the sentence
        # so that the model know when to start and stop predicting.
        w = '<start> ' + w + ' <end>'
        return w

    def _tokenize(self, lang,lang_tokenizer=None):
      if lang_tokenizer==None:
          lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
              filters='')
          lang_tokenizer.fit_on_texts(lang)

      tensor = lang_tokenizer.texts_to_sequences(lang)

      tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                             padding='post')

      return tensor, lang_tokenizer

    def _convert(self, lang,tensor):
        return [lang.index_word[t] if t!=0 else '' for t in tensor]

    def _preprocess(self, sentences, tokenizer):
        """
        Args:
            sentences: a python list of of strings
            tokenizer: keras_preprocessing.text.Tokenizer for mapping words to integers
        """
        sentences = [self._preprocess_sentence(sentence) for sentence in sentences]
        tokens, _ = self._tokenize(sentences,tokenizer)
        return tokens

    def _decode_sequences(self, input_seqs, output_tokenizer, max_decode_length=50):
        """
        Arguments:
            input_seqs: int tensor of shape (BATCH_SIZE,SEQ_LEN)

        """
        # Encode the input as state vectors.
        batch_size = input_seqs.shape[0]
        states_value = self.encoder_model.predict(input_seqs)

        # Populate the first character of target sequence with the start character.
        target_seq = tf.ones([batch_size,1])

        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        decoded_sentences = [[] for _ in range(batch_size)]
        for i in range(max_decode_length):
            output_tokens, decoder_state = self.decoder_model.predict(
                [target_seq,states_value])

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[:, -1, :],axis=-1)
            tokens = self._convert(output_tokenizer,sampled_token_index)
            for j in range (batch_size):
                decoded_sentences[j].append(tokens[j])

            # Update the target sequence (of length 1).
            target_seq = tf.expand_dims(tf.constant(sampled_token_index),axis=-1)

            # Update states
            states_value = decoder_state

        return decoded_sentences
    
    def predict(self, instances, **kwargs):
        machine_translations = self._decode_sequences(
            self._preprocess(instances,self.encoder_tokenizer),
            self.decoder_tokenizer,
            MAX_TRANSLATE_LENGTH
        )
        return machine_translations
    

    @classmethod
    def from_path(cls, model_dir):
        encoder_model = tf.keras.models.load_model(os.path.join(model_dir,'encoder_model.h5'))
        decoder_model = tf.keras.models.load_model(os.path.join(model_dir,'decoder_model.h5'))
    
        encoder_tokenizer = pickle.load(open(os.path.join(model_dir,'encoder_tokenizer.pkl'),'rb'))
        decoder_tokenizer = pickle.load(open(os.path.join(model_dir,'decoder_tokenizer.pkl'),'rb'))

        return cls(encoder_model, encoder_tokenizer, decoder_model, decoder_tokenizer)

Overwriting predictor.py


### Test Predictor Class Works Locally

In [8]:
import predictor

sentences = [
    "El soldado actuó valientemente.", 
    "Su pierna mala le impidió ganar la carrera.",
    "No estamos comiendo.",
    "Está llegando el invierno.",
    "El invierno se acerca.",
    "Hay una bolsa sobre el escritorio.",
    "¿Qué tal si damos un paseo después del almuerzo?"
]

predictor = predictor.TranslatePredictor.from_path(MODEL_PATH)
predictor.predict(sentences)

W0617 19:34:52.350484 139915477219072 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.
W0617 19:34:52.596298 139915477219072 hdf5_format.py:171] No training configuration found in save file: the model was *not* compiled. Compile it manually.


[['the', 'storm', 'were', '<end>', '.', '<end>', '', '', '', '', ''],
 ['her', 'car', 'inspired', 'me', '.', '<end>', '', '', '', '', ''],
 ['we', 're', 'not', 'eating', '.', '<end>', '', '', '', '', ''],
 ['winter', 'is', 'coming', '.', '<end>', '', '', '', '', '', ''],
 ['winter', 'is', 'coming', 'on', '.', '<end>', '', '', '', '', ''],
 ['there', 's', 'a', 'bus', 'here', '.', '<end>', '', '', '', ''],
 ['how', 'about', 'you', 'a', 'sad', '?', '<end>', '', '', '', '']]

## 3. Package Predictor Class and Dependencies

In [4]:
%%writefile setup.py
from setuptools import setup

setup(
    name='translate_custom_predict_code',
    version='0.1',
    scripts=['predictor.py'])

Overwriting setup.py


In [5]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
writing requirements to translate_custom_predict_code.egg-info/requires.txt
writing translate_custom_predict_code.egg-info/PKG-INFO
writing top-level names to translate_custom_predict_code.egg-info/top_level.txt
writing dependency_links to translate_custom_predict_code.egg-info/dependency_links.txt
reading manifest file 'translate_custom_predict_code.egg-info/SOURCES.txt'
writing manifest file 'translate_custom_predict_code.egg-info/SOURCES.txt'

running check


creating translate_custom_predict_code-0.1
creating translate_custom_predict_code-0.1/translate_custom_predict_code.egg-info
creating translate_custom_predict_code-0.1/translate_custom_predict_code.egg-info/.ipynb_checkpoints
copying files to translate_custom_predict_code-0.1...
copying predictor.py -> translate_custom_predict_code-0.1
copying setup.py -> translate_custom_predict_code-0.1
copying translate_custom_predict_code.egg-info/PKG-INFO -> translate_custom_predict_code-0.1/translate_custom_

In [6]:
!gsutil cp dist/translate_custom_predict_code-0.1.tar.gz gs://$BUCKET/translate/predict_code/

Copying file://dist/translate_custom_predict_code-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  2.4 KiB/  2.4 KiB]                                                
Operation completed over 1 objects/2.4 KiB.                                      


## 4. Deploy

Warning: If you get a GCS access error, grant the 'Storage Object Viewer' role on the bucket that contains your artifacts to the service account being used.

In [7]:
!gcloud ai-platform models create $MODEL_NAME --regions us-central1

!gcloud beta ai-platform versions create $VERSION_NAME \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --origin gs://$BUCKET/translate/model/ \
  --package-uris gs://$BUCKET/translate/predict_code/translate_custom_predict_code-0.1.tar.gz \
  --prediction-class predictor.TranslatePredictor

ERROR: (gcloud.ai-platform.models.create) Resource in project [vijays-sandbox] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name
Creating version (this might take a few minutes)......done.


## 5. Invoke API

In [10]:
import googleapiclient.discovery

instances = [
    ["El soldado actuó valientemente."], 
    ["Su pierna mala le impidió ganar la carrera."]
]

service = googleapiclient.discovery.build('ml', 'v1')
name = 'projects/{}/models/{}/versions/{}'.format(PROJECT_ID, MODEL_NAME, VERSION_NAME)

response = service.projects().predict(
    name=name,
    body={'instances': instances}
).execute()

if 'error' in response:
    raise RuntimeError(response['error'])
else:
  print(response['predictions'])

RuntimeError: Prediction failed: unknown error.